In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [ ]:
class SMAcross(Strategy):
    n1 = 50
    n2 = 100

    def init(self):
        close = self.data.Close
        self.ma1 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n1)
        self.ma2 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n2)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()

In [ ]:
# Download historical SPY data
spy = yf.download('SPY', '2010-01-01')

In [ ]:
# Backtest strategy
bt = Backtest(spy, SMAcross, cash=10000, commission=.002, exclusive_orders=True)

In [ ]:
# Save results from backtest
stats = bt.run()
stats

In [ ]:
# Plot the results
bt.plot()


In [37]:
# Optimize the strategys return
optim = bt.optimize(n1=range(10, 250, 10),
                    n2=range(10, 500, 10),
                    # At least 20 periods between signals
                    constraint=lambda param: param.n2 - param.n1 > 10,
                    maximize='Sharpe Ratio')

/opt/homebrew/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 852 configurations.
  output = _optimize_grid()
/opt/homebrew/lib/python3.10/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


In [38]:
optim

Start                     2010-01-04 00:00:00
End                       2022-12-19 00:00:00
Duration                   4732 days 00:00:00
Exposure Time [%]                   82.077206
Equity Final [$]                281703.151636
Equity Peak [$]                 310563.070023
Return [%]                         181.703152
Buy & Hold Return [%]              235.321612
Return (Ann.) [%]                    8.324468
Volatility (Ann.) [%]               16.951689
Sharpe Ratio                          0.49107
Sortino Ratio                        0.741874
Calmar Ratio                         0.244143
Max. Drawdown [%]                  -34.096751
Avg. Drawdown [%]                   -2.074941
Max. Drawdown Duration      958 days 00:00:00
Avg. Drawdown Duration       25 days 00:00:00
# Trades                                    5
Win Rate [%]                             80.0
Best Trade [%]                      88.950981
Worst Trade [%]                    -11.672273
Avg. Trade [%]                    

In [39]:
# Plot optimal strategy
bt.plot()

Row(id='5110', ...)